In [71]:
import sys
import os
os.environ['HEASOFTPYDIR']= os.path.split(os.getcwd())[0]

import heasoftpy as hsp

from astropy.table import Table
from astropy.time import Time
from astropy.io import fits
import astropy.units as u

# Set up the NICER obsid directory

We'll assume the user has downloaded the NICER observation directory for OBSID ``4142010107`` in a directory called ``/Users/mcorcora/tmp/nicer``

That is the directory ``/Users/mcorcora/tmp/nicer/4142010107`` contains the ``xti``, ``auxil`` and ``hk`` directories containing the standard archived data for the observation

In [73]:
nicerdatadir = '/Users/mcorcora/tmp/nicer'
nicerobsID = '4142010107'

In [41]:
cwd = os.getcwd()
os.chdir(nicerdatadir)

# Create the nicerl2 task

In [105]:
tstart = Time.now()
print(f'Start at: {tstart.iso[:19]}')
nicerl2 = hsp.HSPTask('nicerl2')

nicerl2.clobber=True
resl2 = nicerl2(indir=nicerobsID, noprompt=True)

tend = Time.now()
print(f'End at: {tend.iso[:19]}')
print(f'nicerl2 took: {(tend.mjd-tstart.mjd)*86400} seconds')

if resl2.returncode != 0:
    print('\n')
    for o in resl2.output[:]:
        print(o)


Start at: 2021-12-17 20:47:51
End at: 2021-12-17 20:47:51
nicerl2 took: 0.5630620988085866 seconds


Parameter clobber: value "1" is not of boolean type.
Parameter clobber: value "1" is not of boolean type.
ERROR: could not retrieve parameter 'clobber' at /Users/mcorcora/software/heasoft/headas/setup/bin/nicerl2 line 187.
PIL ERROR PIL_BAD_VALUE: problem converting string to other type
Task nicerl2 1.18 terminating with status -3040



In [100]:
resl2.returncode

32

# Analyzing NICER spectra

In [9]:
# get the on-axis rmf
res = quzcif(mission='nicer', instrument='xti',detector='-',
             filter='-',codename='matrix', date='-', time='-',expr='-')
rmf = [x.split()[0] for x in res.output if 'nixtiref'  in x][0]

# get the on-axis arf


In [78]:
resl2.output[70:72]

['ERROR: outfile 4142010107/xti/event_cl/ni4142010107_0mpu0_ufa.evt exists and clobber=NO at /Users/mcorcora/software/heasoft/headas/setup/bin/nimpucal line 158.',
 'Task nimpucal 1.10 terminating with status -1']